In [1]:
import numpy as np
import matplotlib.image as mpimg
import cv2
import sys
import pickle
import os
from datetime import date

In [3]:
#intrinsic calibration with checkerboard
import numpy as np
import matplotlib.image as mpimg
import cv2
import sys
import pickle
import os
from datetime import date

chess_size = (8, 5) #size of chessboard in calibration images

def calibrate(directory, size):
    #This functions return the camera matrix and distortion coefficients by perfroming calibration on a set of chessboard images
    #obtaining files in directory
    cal_files = os.listdir(directory)

    #defining image and object points
    img_points = []
    obj_points = []
    objp = np.zeros((size[0]*size[1],3), np.float32)
    objp[:,:2] = np.mgrid[0:size[0], 0:size[1]].T.reshape(-1,2)

    #iterating over images in directory 
    for file in cal_files:

        #reading image
        if(file[-3:]=="png"):
            cal_img = mpimg.imread(directory + file)

            #converting to grayscale
            gray = cv2.cvtColor(cal_img, cv2.COLOR_RGB2GRAY)
            gray = np.uint8(gray*255)
            # gray = np.uint8(cal_img*255)

            #obtaining corners in chessboard
            ret, corners = cv2.findChessboardCorners(gray, size)
            
            if (ret):
                img = cv2.drawChessboardCorners(cal_img, size, corners, ret)
                cv2.imshow('img',img)
                cv2.waitKey(100)

                img_points.append(corners)
                obj_points.append(objp)

    #performing calibration
    ret, cam_mtx, dist_coef, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, gray.shape[::-1], None, None)

    return cam_mtx, dist_coef, rvecs, tvecs

	
calibrate_img_dir = "../d435_calibration/"
#initialize

mtx = np.ndarray(shape=(3,3)) #setting camera matrix as global variables
dist = np.ndarray(shape=(1,5))  #setting distortion coefficients as global variables


#perform calibration
[mtx, dist, rvecs, tvecs] = calibrate(calibrate_img_dir, chess_size)
print mtx
print dist

#save calibration parameters
calibration_results = {"camera_matrix": mtx,
                        "distortion_coefficients": dist}
calibration_results_dir = 'davis_calibration_' + str(date.today()) + '.pickle'
pickle.dump(calibration_results, open(calibration_results_dir, 'wb'))

[[631.67934587   0.         327.59874164]
 [  0.         634.35127977 217.73236375]
 [  0.           0.           1.        ]]
[[ 8.20953340e-02  1.06112541e+00 -1.34680185e-02  2.88612993e-03
  -4.77827126e+00]]


In [1]:
#intrinsic calibration with charuco
import numpy as np
import matplotlib.image as mpimg
import cv2
from cv2 import aruco
import sys
import pickle
import os
from datetime import date

#Charuco properties
aruco_params = aruco.DetectorParameters_create()
aruco_params.cornerRefinementMethod = aruco.CORNER_REFINE_SUBPIX
aruco_params.cornerRefinementWinSize = 5
aruco_params.cornerRefinementMinAccuracy = 0.001
aruco_params.cornerRefinementMaxIterations = 5
# charuco_dict = aruco.Dictionary_get(aruco.DICT_5X5_250)
# CHARUCO_BOARD = aruco.CharucoBoard_create(
#         squaresX=18,
#         squaresY=18,
#         squareLength=0.015,
#         markerLength=0.01,
#         dictionary=charuco_dict)
charuco_dict = aruco.Dictionary_get(aruco.DICT_4X4_1000)
CHARUCO_BOARD = aruco.CharucoBoard_create(
        squaresX=8,
        squaresY=12,
        squareLength=0.023,
        markerLength=0.020,
        dictionary=charuco_dict)


def calibrate_charuco(directory):
    #This functions return the camera matrix and distortion coefficients by perfroming calibration on a set of chessboard images
    #obtaining files in directory
    cal_files = os.listdir(directory)

    #defining image and object points
    img_points = []
    obj_points = []

    corners_all = []
    ids_all = []

    #iterating over images in directory 
    for file in cal_files:

        #reading image
        if(file[-3:]=="png"):
            cal_img = cv2.imread(directory + file)

            #converting to grayscale
            # gray = cv2.cvtColor(cal_img, cv2.COLOR_RGB2GRAY)
            # gray = np.uint8(gray*255)
            gray = np.uint8(cal_img)

            #obtaining corners in charuco
            corners, ids, _ = aruco.detectMarkers(
                        image=gray,
                        dictionary=charuco_dict,
                        parameters=aruco_params)

            response, chararuco_corners, chararuco_ids = aruco.interpolateCornersCharuco(
                markerCorners=corners,
                markerIds=ids,
                image=gray,
                board=CHARUCO_BOARD)

            if response > 3:
                corners_all.append(chararuco_corners)
                ids_all.append(chararuco_ids)


    #performing calibration

    mtx = np.ndarray(shape=(3,3)) #setting camera matrix as global variables
    dist = np.ndarray(shape=(1,5))  #setting distortion coefficients as global variables
    calibration, cam_mtx, dist_coef, rvecs, tvecs = aruco.calibrateCameraCharuco(
                charucoCorners=corners_all,
                charucoIds=ids_all,
                board=CHARUCO_BOARD,
                imageSize=(346, 260),
                cameraMatrix=None,
                distCoeffs=None)

    return cam_mtx, dist_coef, rvecs, tvecs

	
calibrate_img_dir = "../davis346_calibration/"
#initialize

mtx = np.ndarray(shape=(3,3)) #setting camera matrix as global variables
dist = np.ndarray(shape=(1,5))  #setting distortion coefficients as global variables


#perform calibration
[mtx, dist, rvecs, tvecs] = calibrate_charuco(calibrate_img_dir)
print mtx
print dist

#save calibration parameters
calibration_results = {"camera_matrix": mtx,
                        "distortion_coefficients": dist}
calibration_results_dir = 'davis346_calibration_' + str(date.today()) + '.pickle'
pickle.dump(calibration_results, open(calibration_results_dir, 'wb'))

[[261.41923524   0.         161.79457156]
 [  0.         261.75504463 123.33115029]
 [  0.           0.           1.        ]]
[[-0.39850654  0.23827918 -0.00051233  0.00040576 -0.09574713]]


In [4]:
import numpy as np
import cv2
import matplotlib.image as mpimg
import pickle
import os
from scipy.spatial.transform import Rotation as R

#Extrensic calibration with checker board

img_dir = '../'
calibration_filemame = '../d435_calibration_2021-03-02.pickle'
pose_data_filename = '../d435_calibration_data2021-01-21.pickle'

chessboard_size = (8, 5)

calibration_file = open(calibration_filemame, 'rb')
calibration_results = pickle.load(calibration_file)

mtx = calibration_results['camera_matrix']
dist = calibration_results['distortion_coefficients']

pose_data_file = open(pose_data_filename, 'rb')
pose_data = pickle.load(pose_data_file)

object_points = np.zeros((chessboard_size[0]*chessboard_size[1],3), np.float32)
object_points[:,:2] = 3*np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1,2)


def draw(img, corners, imgpts):
    corner = tuple(corners[0].ravel())
    img = cv2.line(img, corner, tuple(imgpts[0].ravel()), (255,0,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[1].ravel()), (0,255,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[2].ravel()), (0,0,255), 5)
    return img

B_T0 = [] #transform from base to tool0
C_AR = [] #transform from camera to aruco

#iterating over images in directory 
for pose_step in pose_data:
    print(pose_step['ee_pose'])
    image_file = pose_step['image_dir']
    
    #reading image
    if(image_file[-3:]=="png"):
        #load image
        img = mpimg.imread(img_dir + image_file)

        #converting to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        gray = np.uint8(gray*255)
        # gray = np.uint8(img*255)
        cv2.imshow('img',gray)
        k = cv2.waitKey(0) & 0xFF

        #obtaining corners in chessboard
        ret, corners = cv2.findChessboardCorners(gray, chessboard_size)
        if ret:
            #refine corner estimation
            criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
            corners2 = cv2.cornerSubPix(gray, corners, (5,5), (-1,-1),criteria)

            # Find the rotation and translation vectors.
            ret,rvecs, tvecs = cv2.solvePnP(object_points, corners2, mtx, dist)

            rot_mat = cv2.Rodrigues(rvecs)

            axis = np.float32([[10,0,0], [0,10,0], [0,0,10]]).reshape(-1,3)

            # project 3D points to image plane
            imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)

            img = draw(img,corners2,imgpts)
            cv2.imshow('img',img)
            k = cv2.waitKey(0) & 0xFF
            if k == ord('s'):
                cv2.imwrite('annotated'+'.png', img)

            B_T0.append(pose_step['ee_pose'])

            p_cam_board = np.array(tvecs/100).reshape(3,-1)
            rotv_cam_board = np.array(rvecs).reshape(3)
            R_cam_board = R.from_rotvec(rotv_cam_board)
            DCM_cam_board = R_cam_board.as_dcm()
            C_AR.append(np.vstack([np.c_[DCM_cam_board, p_cam_board], [0, 0, 0, 1]]))



            print(tvecs)
            print(rvecs)


[[-9.99999989e-01 -1.00774547e-04  1.09043311e-04 -1.49972876e-01]
 [-1.09058589e-04  1.51611869e-04 -9.99999983e-01 -5.69039443e-01]
 [ 1.00758013e-04 -9.99999983e-01 -1.51622858e-04  6.99929151e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[-7.11495488]
 [-2.94999806]
 [63.47220117]]
[[-0.03759433]
 [-0.00409538]
 [ 0.05255886]]
[[-9.99999999e-01  6.63064898e-06  3.76710246e-05 -1.75464466e-01]
 [-3.68207889e-05  9.98408298e-02 -9.95003421e-01 -6.56659884e-01]
 [-1.03586248e-05 -9.95003421e-01 -9.98408295e-02  6.08242219e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[-9.68233761]
 [-0.06482585]
 [53.78450842]]
[[ 0.07397873]
 [-0.00021661]
 [ 0.05167192]]
[[-0.99827341  0.05868887  0.002413   -0.14270423]
 [ 0.00234838  0.08092519 -0.99671741 -0.5847015 ]
 [-0.05869149 -0.99499082 -0.08092329  0.75164295]
 [ 0.          0.          0.          1.        ]]
[[-2.22990098]
 [-6.68490343]
 [69.6451124 ]]
[[0.03951076]
 [0.05041696]

In [1]:
#Extrensic calibration with aruco
import numpy as np
import cv2
import matplotlib.image as mpimg
import pickle
from cv2 import aruco

img_file = '../images/test_aruco.png'
calibration_file = '../calibration_2021-01-09.pickle'

#ARUCO properties
aruco_dict = aruco.Dictionary_get(aruco.DICT_6X6_250)
size_of_marker = 0.06

my_file = open(calibration_file, 'rb')
calibration_results = pickle.load(my_file)

mtx = calibration_results['camera_matrix']
dist = calibration_results['distortion_coefficients']

#load image
img = mpimg.imread(img_file)

#converting to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
gray = np.uint8(gray*255)

corners = []
parameters =  aruco.DetectorParameters_create()
corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

frame_markers = aruco.drawDetectedMarkers(img.copy(), corners, ids)

rvecs, tvecs, trash = aruco.estimatePoseSingleMarkers(corners, size_of_marker, mtx, dist)

cv2.imshow('img',frame_markers)
k = cv2.waitKey(0) & 0xFF
if k == ord('s'):
    cv2.imwrite('annotated_aruco'+'.png', frame_markers)

print(tvecs[0])
print(rvecs[0])
print(trash)



TypeError: 'NoneType' object has no attribute '__getitem__'

In [2]:
import numpy as np
import cv2
from cv2 import aruco
import matplotlib.image as mpimg
import pickle
import os
from scipy.spatial.transform import Rotation as R

#Extrensic calibration with Charuco

img_dir = '../'
calibration_filemame = 'davis346_calibration_2021-04-25.pickle'
pose_data_filename = '../davis346_calibration_data2021-04-25.pickle'

#Charuco properties
aruco_params = aruco.DetectorParameters_create()
aruco_params.cornerRefinementMethod = aruco.CORNER_REFINE_SUBPIX
aruco_params.cornerRefinementWinSize = 7
aruco_params.cornerRefinementMinAccuracy = 0.001
aruco_params.cornerRefinementMaxIterations = 100
# charuco_dict = aruco.Dictionary_get(aruco.DICT_5X5_250)
# CHARUCO_BOARD = aruco.CharucoBoard_create(
#         squaresX=18,
#         squaresY=18,
#         squareLength=0.015,
#         markerLength=0.01,
#         dictionary=charuco_dict)
charuco_dict = aruco.Dictionary_get(aruco.DICT_4X4_1000)
CHARUCO_BOARD = aruco.CharucoBoard_create(
        squaresX=8,
        squaresY=12,
        squareLength=0.023,
        markerLength=0.020,
        dictionary=charuco_dict)

calibration_file = open(calibration_filemame, 'rb')
calibration_results = pickle.load(calibration_file)

mtx = calibration_results['camera_matrix']
dist = calibration_results['distortion_coefficients']

pose_data_file = open(pose_data_filename, 'rb')
pose_data = pickle.load(pose_data_file)

B_T0 = [] #transform from base to tool0
C_AR = [] #transform from camera to aruco

#iterating over images in directory 
for pose_step in pose_data:
    print(pose_step['ee_pose'])
    image_file = pose_step['image_dir']
    
    #reading image
    if(image_file[-3:]=="png"):
        #load image
        img = mpimg.imread(img_dir + image_file)

        #converting to grayscale
        # gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        # gray = np.uint8(gray*255)
        gray = np.uint8(img*255)

        cv2.imshow('img',gray)
        k = cv2.waitKey(0) & 0xFF

        corners, ids, _ = aruco.detectMarkers(
                        image=gray,
                        dictionary=charuco_dict,
                        parameters=aruco_params)

        response, chararuco_corners, chararuco_ids = aruco.interpolateCornersCharuco(
            markerCorners=corners,
            markerIds=ids,
            image=gray,
            board=CHARUCO_BOARD) 

        if response > 4:           
            rvec = np.empty(shape=(1,))
            tvec = np.empty(shape=(1,))
            retval, rvecs, tvecs = aruco.estimatePoseCharucoBoard(chararuco_corners, chararuco_ids, CHARUCO_BOARD, mtx, dist, rvec, tvec)

            img = aruco.drawDetectedCornersCharuco(
                        image=img,
                        charucoCorners=chararuco_corners,
                        charucoIds=chararuco_ids)

            cv2.imshow('img',img)
            k = cv2.waitKey(0) & 0xFF
            if k == ord('s'):
                cv2.imwrite('annotated'+'.png', img)

            B_T0.append(pose_step['ee_pose'])


            aruco_correction_dcm = np.array([[0, 1, 0], [1, 0, 0], [0, 0, -1]])#Correct alignment of charuco frame

            p_cam_board = np.array(tvecs).reshape(3,-1)
            rotv_cam_board = np.array(rvecs).reshape(3)
            R_cam_board = R.from_rotvec(rotv_cam_board)
            DCM_cam_board = np.matmul(R_cam_board.as_dcm(), aruco_correction_dcm)
            C_AR.append(np.vstack([np.c_[DCM_cam_board, p_cam_board], [0, 0, 0, 1]]))


            print(tvecs)
            print(rvecs)


]]
[[ 2.11719569]
 [-2.28164681]
 [ 0.37808401]]
[[-0.97472307 -0.07062732 -0.21195926 -0.04841168]
 [-0.03802416  0.98731909 -0.15412716 -0.47907092]
 [ 0.22015701 -0.14217172 -0.96504823  0.29025769]
 [ 0.          0.          0.          1.        ]]
[[0.06184215]
 [0.10493171]
 [0.2448962 ]]
[[-2.24558551]
 [ 2.05961663]
 [-0.38176568]]
[[-9.63040068e-01  8.80414596e-02 -2.54563408e-01  2.63381375e-04]
 [ 1.08560872e-01  9.91782787e-01 -6.76863382e-02 -5.19217812e-01]
 [ 2.46512403e-01 -9.28202812e-02 -9.64684420e-01  2.82275339e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[0.08498839]
 [0.10494002]
 [0.23980329]]
[[-2.02699143]
 [ 2.17935821]
 [-0.33663735]]
[[-9.64795058e-01 -3.23121187e-04 -2.63002646e-01  3.56678297e-02]
 [ 1.93859326e-03  9.99963342e-01 -8.34004744e-03 -4.89414927e-01]
 [ 2.62995699e-01 -8.55629171e-03 -9.64759064e-01  3.03715964e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[0.12131906]
 [0.0655824 ]
 [

In [3]:
print(C_AR[0])
print(B_T0[0])

[[-0.99966936  0.01020538 -0.02360121  0.11718296]
 [-0.00964317 -0.9996699  -0.02381392  0.09213445]
 [-0.02383645 -0.02357846  0.99943778  0.11777793]
 [ 0.          0.          0.          1.        ]]
[[-0.99947185  0.01929564 -0.02614782 -0.04882188]
 [ 0.01945147  0.99979445 -0.00571853 -0.51811011]
 [ 0.0260321  -0.00622412 -0.99964173  0.19966494]
 [ 0.          0.          0.          1.        ]]


In [4]:
#Solve the rotation
def solve_transform(X, Y):

        def ralign(X, Y):

            m, n = X.shape
            mx = X.mean(1)
            my = Y.mean(1)
            Xc = X - np.tile(mx, (n, 1)).T
            Yc = Y - np.tile(my, (n, 1)).T

            sx = np.mean(np.sum(Xc * Xc, 0))
            sy = np.mean(np.sum(Yc * Yc, 0))

            Sxy = np.dot(Yc, Xc.T) / n

            U, D, V = np.linalg.svd(Sxy, full_matrices=True, compute_uv=True)
            V = V.T.copy()

            # r = np.rank(Sxy)
            r = np.ndim(Sxy)
            d = np.linalg.det(Sxy)
            S = np.eye(m)
            if r > (m - 1):
                if (np.det(Sxy) < 0):
                    S[m, m] = -1;
                elif (r == m - 1):
                    if (np.det(U) * np.det(V) < 0):
                        S[m, m] = -1
                else:
                    R = np.eye(2)
                    c = 1
                    t = np.zeros(2)
                    return R, c, t

            R = np.dot(np.dot(U, S), V.T)
            c = np.trace(np.dot(np.diag(D), S)) / sx
            t = my - c * np.dot(R, mx)

            return R, t

        R, T = ralign(X[0:3, :], Y[0:3, :])
        H = np.zeros((4, 4), dtype=float)
        H[3, 3] = 1
        for j in range(0, 3):
            H[j, 3] = T[j]
            for i in range(0, 3):
                H[j, i] = R[j, i]

        return H

def solveTransformation(H_TBs, H_ACs, initial_T0_C):
    [N, L] = [30, 0.5]

    X_A = np.zeros([4, 3 * N])
    X_A[0, 0:N] = np.linspace(0, L, N + 1)[1:]
    X_A[1, N:2 * N] = np.linspace(0, L, N + 1)[1:]
    X_A[2, 2 * N:3 * N] = np.linspace(0, L, N + 1)[1:]
    X_A[3, :] = 1

    solverBreakIterations = 5000
    solverMinRMSE = 1e-9
    solverRMSE = 1e3

    H_CT = initial_T0_C

    for solverIterations in range(0, solverBreakIterations):

        xyzlist = []
        colorlist = []

        # solve transform: X_A -> X_B

        for i in np.arange(0, len(H_TBs)):

            X_Bi = np.dot(H_TBs[i], np.dot(H_CT, np.dot(H_ACs[i], X_A)))
            X_Ai = X_A

            for j in range(0, N * 3):
                xyzlist.append(X_Bi[0:3, j])
                if j < N:
                    color = [255, 0, 0]
                elif N <= j < 2 * N:
                    color = [0, 255, 0]
                else:
                    color = [0, 0, 255]
                colorlist.append(color)

            if (i == 0):
                X_Bj = X_Bi
                X_Aj = X_Ai
            else:
                X_Bj = np.append(X_Bj, X_Bi, axis=1)
                X_Aj = np.append(X_Aj, X_Ai, axis=1)

        H_AB = solve_transform(X_Aj, X_Bj)

        X_B = np.dot(H_AB, X_A)

        if solverIterations <> 0:

            solverRMSE = np.average(np.linalg.norm((X_B - X_B_old)[0:3, :], axis=0))
            print('Converging RMSE: {:.2e}'.format(solverRMSE))

        X_B_old = np.copy(X_B)

        # solve transform: X_C -> X_T

        for i in np.arange(0, len(H_TBs)):

            X_Ti = np.dot(np.linalg.inv(H_TBs[i]), X_B)
            X_Ci = np.dot(H_ACs[i], X_A)

            if (i == 0):
                X_Tj = X_Ti
                X_Cj = X_Ci
            else:
                X_Tj = np.append(X_Tj, X_Ti, axis=1)
                X_Cj = np.append(X_Cj, X_Ci, axis=1)

        H_CT = solve_transform(X_Cj, X_Tj)

        solverIterations += 1

        count = len(xyzlist)

        if solverRMSE < solverMinRMSE:

            print("calibrarion Done")
            print(H_CT)
    return H_CT

In [5]:
#solve calibration
from datetime import date

#Drill EE
# initial_translation = np.array([0.17019012, 0.01950491, -0.00239074]).reshape(3,-1)
# initial_dcm = np.array([[-0.70284836,    0.02252686,     0.71098293],
#                         [0.71096759,    -0.01007902,    0.70315254],
#                         [0.02300584,    0.99969543,     -0.00893185]])
# initial_transformation = np.vstack([np.c_[initial_dcm, initial_translation], [0, 0, 0, 1]])


#d435 holder
# initial_transformation = np.array([[1.0,        0.0,     0.0,     -0.1],
#                         [0.0,       0.0,    1.0,     0.0],
#                         [0.0,       -1.0,     0,       0.05],
#                         [0.,        0.,      0.,             1.]])


#d435 holder - davis
initial_transformation = np.array([[1.0,        0.0,     0.0,     0.0],
                        [0.0,       0.0,    1.0,     0.0],
                        [0.0,       -1.0,     0.0,       0.05],
                        [0.,        0.,      0.,             1.]])
                        


calibrated_transform = solveTransformation(B_T0, C_AR, initial_transformation)

calibration_results = {"camera_matrix": mtx,
                        "distortion_coefficients": dist,
                        "extrensic_calibration": calibrated_transform}
calibration_results_dir = 'davis346_full_calibration_results_' + str(date.today()) + '.pickle'
pickle.dump(calibration_results, open(calibration_results_dir, 'wb'))

4454]
 [-0.01343114  0.99988961  0.00635346 -0.19752954]
 [-0.00396031 -0.00640718  0.99997163  0.08993638]
 [ 0.          0.          0.          1.        ]]
Converging RMSE: 1.31e-16
calibrarion Done
[[ 0.99990196  0.0134056   0.00404593 -0.00224454]
 [-0.01343114  0.99988961  0.00635346 -0.19752954]
 [-0.00396031 -0.00640718  0.99997163  0.08993638]
 [ 0.          0.          0.          1.        ]]
Converging RMSE: 8.89e-17
calibrarion Done
[[ 0.99990196  0.0134056   0.00404593 -0.00224454]
 [-0.01343114  0.99988961  0.00635346 -0.19752954]
 [-0.00396031 -0.00640718  0.99997163  0.08993638]
 [ 0.          0.          0.          1.        ]]
Converging RMSE: 1.64e-16
calibrarion Done
[[ 0.99990196  0.0134056   0.00404593 -0.00224454]
 [-0.01343114  0.99988961  0.00635346 -0.19752954]
 [-0.00396031 -0.00640718  0.99997163  0.08993638]
 [ 0.          0.          0.          1.        ]]
Converging RMSE: 1.43e-16
calibrarion Done
[[ 0.99990196  0.0134056   0.00404593 -0.00224454]
 [

In [6]:
#Dump data to json to solve with Matlab
import h5py

hf = h5py.File('davis346_extrensic_calibration_data_04_25.h5', 'w')

hf.create_dataset('C_AR', data=C_AR)
hf.create_dataset('B_T0', data=B_T0)
hf.create_dataset('initial_transformation', data=initial_transformation)
hf.create_dataset('python_result', data=calibrated_transform)

hf.close()


In [2]:
import numpy as np
[N, L] = [30, 0.5]

X_A = np.zeros([4, 3 * N])
X_A[0, 0:N] = np.linspace(0, L, N + 1)[1:]
X_A[1, N:2 * N] = np.linspace(0, L, N + 1)[1:]
X_A[2, 2 * N:3 * N] = np.linspace(0, L, N + 1)[1:]
X_A[3, :] = 1

print(X_A)

[[0.01666667 0.03333333 0.05       0.06666667 0.08333333 0.1
  0.11666667 0.13333333 0.15       0.16666667 0.18333333 0.2
  0.21666667 0.23333333 0.25       0.26666667 0.28333333 0.3
  0.31666667 0.33333333 0.35       0.36666667 0.38333333 0.4
  0.41666667 0.43333333 0.45       0.46666667 0.48333333 0.5
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.

In [3]:
g = [1.57, 0, 0]

from scipy.spatial.transform import Rotation as R
rot = R.from_rotvec(g)
print(rot.as_dcm())

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  7.96326711e-04 -9.99999683e-01]
 [ 0.00000000e+00  9.99999683e-01  7.96326711e-04]]


In [4]:
import numpy as np

TCP_to_pressure_foot = np.array([[-0.7065235,   -0.0149079,     0.7075326,  0.10418],
                            [0.7076895,         -0.0155756,     0.7063519,  0.10444],
                            [0.0004900,         0.9997675,      0.0215547,  0.07405],
                            [0,                 0,              0,          1]])


correction = np.array([0.0011,0.0025, 0]).reshape(3,-1)

TCP_to_pressure_foot[:3, 3] = TCP_to_pressure_foot[:3, 3] + np.matmul(TCP_to_pressure_foot[:3, :3], correction).reshape(3)
print(TCP_to_pressure_foot)

[[-7.06523500e-01 -1.49079000e-02  7.07532600e-01  1.03365554e-01]
 [ 7.07689500e-01 -1.55756000e-02  7.06351900e-01  1.05179519e-01]
 [ 4.90000000e-04  9.99767500e-01  2.15547000e-02  7.65499578e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [3]:
import pickle

g = pickle.load(open('../d435_full_calibration_results_2021-01-21.pickle','rb'))

print(g['extrensic_calibration'])

[[ 0.99969604  0.02116629  0.01264188 -0.20036064]
 [-0.01348879  0.04035957  0.99909417  0.01311723]
 [ 0.0206369  -0.99896101  0.04063281  0.01482989]
 [ 0.          0.          0.          1.        ]]
